In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [8]:
# conf = pyspark.SparkConf().setAppName("APITest").setMaster("local")
# sc = SparkSession.builder.config(conf=conf).getOrCreate()
sc = SparkSession.builder.appName("master").config("master", "local").getOrCreate()

25/06/12 15:13:18 WARN Utils: Your hostname, shasankperiwal resolves to a loopback address: 127.0.1.1; using 10.10.66.70 instead (on interface wlp0s20f3)
25/06/12 15:13:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/12 15:13:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
import requests, json

In [9]:
data=[
    (1, "40.7128", "-74.0060", None),
    (2, "34.0522", "-118.2437", "Los Angels"),
    (3, "51.5074", "-0.1278", None)
]

columns = ["user_id", "latitude", "longitude", "city"]

df = sc.createDataFrame(data=data, schema=columns)
df.show()

+-------+--------+---------+----------+
|user_id|latitude|longitude|      city|
+-------+--------+---------+----------+
|      1| 40.7128| -74.0060|      null|
|      2| 34.0522|-118.2437|Los Angels|
|      3| 51.5074|  -0.1278|      null|
+-------+--------+---------+----------+



In [10]:
res = requests.get("https://geocode.maps.co/reverse?lat=40.7128&lon=-74.0060&api_key=67b4b53293354292471295bege5575c").json()["address"]["city"]
print(res)

New York


In [17]:
def my_func(lat, long):
    res = requests.get(f"https://geocode.maps.co/reverse?lat={lat}&lon={long}&api_key=67b4b53293354292471295bege5575c").json()["address"]["city"]
    return

# if we assume that my_func returns a string
my_udf = UserDefinedFunction(my_func)

df = df.withColumn('newcity', when(df.city.isNull(), my_udf(df.latitude, df.longitude)).otherwise(df.city))
df.select("user_id", "latitude", "longitude", col("newcity").alias("city")).show()

+-------+--------+---------+----------+
|user_id|latitude|longitude|      city|
+-------+--------+---------+----------+
|      1| 40.7128| -74.0060|      null|
|      2| 34.0522|-118.2437|Los Angels|
|      3| 51.5074|  -0.1278|      null|
+-------+--------+---------+----------+

